In [1]:
import os
import struct
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl

In [2]:
# save the original binary MNIST data files in 0-255
def read(dataset = "training", path = "."):
    if dataset is "training":
        fname_img = os.path.join(path, 'train-images-idx3-ubyte')
        fname_lbl = os.path.join(path, 'train-labels-idx1-ubyte')
    elif dataset is "testing":
        fname_img = os.path.join(path, 't10k-images-idx3-ubyte')
        fname_lbl = os.path.join(path, 't10k-labels-idx1-ubyte')
    else:
        raise ValueError( "It needs to be between 'testing' and 'training'")

    with open(fname_lbl, 'rb') as flbl:
        magic, num = struct.unpack(">II", flbl.read(8))
        lbl = np.fromfile(flbl, dtype=np.int8)
        # print(len(lbl))
    with open(fname_img, 'rb') as fimg:
        magic, num, rows, cols = struct.unpack(">IIII", fimg.read(16))
        image = np.fromfile(fimg, dtype=np.uint8)      
        image = image.reshape(len(lbl), rows, cols)

    get_image = lambda idx: (lbl[idx], image[idx])

    for i in range(len(lbl)):
        yield get_image(i)

training_data = list(read(dataset = "training",path = r'C:\Users\Han\Desktop\Box Sync\CS 559\hwk2'))
testing_data = list(read(dataset = "testing",path = r'C:\Users\Han\Desktop\Box Sync\CS 559\hwk2'))
training_label = np.zeros((len(training_data),1))
training_desiredout =np.zeros((len(training_data),10))
training_image = np.zeros((len(training_data),28*28))
testing_label = np.zeros((len(testing_data),1))
testing_desiredout = np.zeros((len(testing_data),10))
testing_image = np.zeros((len(testing_data),28*28))

# split the training and testing data to labels and images
for i in range(len(training_data)):
    temp = training_data[i]
    training_label[i] = temp[0]
    training_desiredout[i,temp[0]] = 1 
    training_image[i,] = temp[1].reshape(1,28*28)
#training_label = training_label.reshape((1,60000))
for i in range(len(testing_data)):
    temp = testing_data[i]
    testing_label[i] = temp[0]
    testing_desiredout[i,temp[0]] = 1 
    testing_image[i,] = temp[1].reshape(1,28*28)
#testing_label = testing_label.reshape((1,10000))

# Rename data
X_train = training_image
X_test = testing_image
Y_train = training_desiredout
Y_test = testing_desiredout
# Standardize X (images from 0-255 to 0-1)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

In [4]:
## initialization
np.random.seed(2)
# use first n samples from training data to train the NN
n = 60000
# number of neurons in 1st hidden layer, N1
N1 = 250
# learning rate
eta = 0.03
# convergence threshold
epsilon = 5e-8
# epoch number 
epoch = 2
# batch size
bs = 1
# initialize errors
entropy = list()
entropy.append(1e1)
entropy.append(5e0)
# initialize real outputs given the current w
out = np.zeros((n,10))
# initialize weights from input to 1st hidden layer
u = np.random.uniform(-0.05,0.05,size = (784*N1)).reshape(784,N1)
# initialize weights from 1st hidden layer to output
v = np.random.uniform(-0.2,0.2,size = (10*N1)).reshape(N1,10)
# initialzie the bias
b = np.random.normal(-0.1,0.1,size = (bs*N1)).reshape(bs,N1)
# initialize condi
m = 0
l = 0
# intialize momentum
momentum = 0.82

# define softmax actiavtion function
def softmax(w):
    e = np.exp(np.array(w) - np.max(w))
    dist = e / np.sum(e)
    return dist
# define a continuous vector to 0-1 vector functin
def vec_to_01(x):
    s1 = np.zeros(x.shape[0])
    q = np.argmax(x)
    s1[q] =1
    return s1
# define softmax's Jacobian matrix function
def softmax_jacobian(s):
    return np.diagflat(s) - np.outer(s, s)
# define tanh's Jacobian matrix function
def tanh_jacobian(t):
    return np.diagflat(1-t**2)
def cross_S(out, Label):
    return - np.sum(np.multiply(out, np.log(Label)) + np.multiply((1-out), np.log(1-Label)))

In [5]:

while (np.abs(entropy[-1]-entropy[-2])>=epsilon):
    m += 1
    index = np.random.choice(np.arange(n), size= n,  replace=False)
    X_train_rndm = X_train[index]
    # re-initialzie last epoch's gradients
    grad_uold = 0
    grad_bold = 0
    grad_vold = 0
    # this loop is where we update the weights
    for i in range(0,n, bs): 
        lf1 = np.dot(X_train_rndm[i:(i+bs)],u) + b  #compute the input layers' induced local field
        z0 = np.tanh(lf1)          #compute the input layer's out
        lf2 = np.dot(z0,v)             #compute the 1st H layers' induced local field
        out[i:(i+bs)] = softmax(lf2)        #compute the 1st H layer's out
        if (True):
            # update u:   
            temp1 = 2 / n * eta * np.sum(Y_train[i:(i+bs)] / out[i:(i+bs)], axis = 0).dot(softmax_jacobian(lf2))
            temp2 = np.dot(temp1,v.T)
            temp2 = temp2.dot(tanh_jacobian(lf1))
            grad_u= np.outer(X_train_rndm[i:(i+bs)],temp2)
            grad_u = grad_uold*momentum+grad_u
            u -= grad_u
            grad_uold = grad_u
           
            # update b:   
            grad_b = (temp2)
            grad_b = grad_bold*momentum+grad_b
            b -=grad_b
            grad_bold = grad_b
           
            # update v:  
            grad_v = np.outer(z0, temp1)
            grad_v = grad_vold*momentum+grad_v
            v -=grad_v
            grad_vold = grad_v
            #if (i%1000 == 0):
                 #print(np.mean(u**2),np.mean(b**2),np.mean(v**2),i)
           
            if np.isnan(v).sum ()>0:
                break
            
    #  calculate mse for the current epoch
    entropy.append(-np.sum(np.log(out) * Y_train)/n)
    epoch +=1

    if (epoch-2) % 2 == 0:
        print('The current epoch is {}, entropy is {}, eta is {}.'.format((epoch-2) ,round(entropy[epoch-2],7) , eta))
        if entropy[epoch-2] <= entropy[epoch-1]:
            eta *= 0.5
            
            
# delete the 2 dummpy points
epoch -= 2
del(entropy[0:2])         

The current epoch is 2, entropy is 2.3293869, eta is 0.001.
The current epoch is 4, entropy is 2.3156306, eta is 0.001.
The current epoch is 6, entropy is 2.3145826, eta is 0.001.
The current epoch is 8, entropy is 2.3144166, eta is 0.0005.
The current epoch is 10, entropy is 2.3126246, eta is 0.0005.
The current epoch is 12, entropy is 2.3125176, eta is 0.00025.
The current epoch is 14, entropy is 2.3135191, eta is 0.00025.
The current epoch is 16, entropy is 2.3121888, eta is 0.00025.


KeyboardInterrupt: 

In [6]:
u0 = u
b0 = b
v0 = v
#  Initialize errors = 0.
error_test = 0
# loop on all testing samples
for i in range(10000):
    lf1 = np.dot(X_train_rndm[i:(i+bs)],u) + b  #compute the input layers' induced local field
    z0 = np.tanh(lf1)          #compute the input layer's out
    lf2 = np.dot(z0,v)             #compute the 1st H layers' induced local field
    out[i:(i+bs)] = softmax(lf2)        #compute the 1st H layer's out
    # Find the largest component of v0 = [v0', v1', ...v9']^T^
    predic_out = np.argmax(out[i,:])
    #  If the prediceted output is different to the testing label, error +=1
    diff = predic_out - np.argmax(Y_test[i,:])
    if diff != 0:
        error_test += 1
error_test/10000

0.9019